# Building a One-Liner Research Agent

Research tasks consume hours of expert time: market analysts manually gathering competitive intelligence, legal teams tracking regulatory changes, engineers investigating bug reports across documentation. The core challenge isn't finding information but knowing what to search for next based on what you just discovered.

The Claude Agent SDK makes it possible to build agents that autonomously explore external systems without a predefined workflow. Unlike traditional workflow automations that follow fixed steps, research agents adapt their strategy based on what they find--following promising leads, synthesizing conflicting sources, and knowing when they have enough information to answer the question.

## By the end of this cookbook, you'll be able to:

- Build a research agent that autonomously searches and synthesizes information with a few lines of code

This foundation applies to any task where the information needed isn't available upfront: competitive analysis, technical troubleshooting, investment research, or literature reviews.

# Why Research Agents?

Research is an ideal agentic use case for two reasons:

1. **Information isn't self-contained**. The input question alone doesn't contain the answer. The agent must interact with external systems (search engines, databases, APIs) to gather what it needs.
2. **The path emerges during exploration**. You can't predetermine the workflow. Whether an agent should search for company financials or regulatory filings depends on what it discovers about the business model. The optimal strategy reveals itself through investigation.

In its simplest form, a research agent searches the web and synthesizes findings. Below, we'll build exactly that with the Claude Agent SDK's built-in web search tool in just a few lines of code.

Note: You can also view the full list of [Claude Code's built-in tools](https://docs.claude.com/en/docs/claude-code/settings#tools-available-to-claude)

# Prerequisites

Before following this guide, ensure you have:

**Required Knowledge**

* Python fundamentals - comfortable with async/await, functions, and basic data structures
* Basic understanding of agentic patterns - we recommend reading [Building effective agents](https://www.anthropic.com/engineering/building-effective-agents) first if you're new to agents

**Required Tools**

* Python 3.11 or higher
* Anthropic API key [(get one here)](https://console.anthropic.com)

**Recommended:**
* Familiarity with the Claude Agent SDK concepts
* Understanding of tool use patterns in LLMs


## Setup

First, install the required dependencies:

In [1]:
%%capture
%pip install -U claude-agent-sdk python-dotenv

Note: Ensure your .env file contains:

```bash
ANTHROPIC_API_KEY=your_key_here
```

Load your environment variables and configure the client:

In [2]:
from dotenv import load_dotenv

load_dotenv()

MODEL = "claude-sonnet-4-5"

## Building Your First Research Agent

Let's start with the simplest possible implementation: a research agent that can search the web and synthesize findings. With the Claude Agent SDK, this takes just a few lines of code.

The key is the query() function, which creates a stateless agent interaction. We'll provide Claude with a single tool, WebSearch, and let it autonomously decide when and how to use it based on our research question.

In [3]:
from claude_agent_sdk import ClaudeAgentOptions, query
from utils.agent_visualizer import print_activity, print_final_result

messages = []
async for msg in query(
    prompt="Research the latest trends in AI agents and give me a brief summary",
    options=ClaudeAgentOptions(model="claude-sonnet-4-5", allowed_tools=["WebSearch"]),
):
    print_activity(msg)
    messages.append(msg)

🤖 Thinking...
🤖 Using: WebSearch()
🤖 Using: WebSearch()
✓ Tool completed
✓ Tool completed
🤖 Thinking...


In [4]:
print_final_result(messages)


📝 Final Result:
## Brief Summary: Latest Trends in AI Agents

Based on the latest research, here are the key trends shaping AI agents in 2025:

**🚀 Market Growth & Adoption**
- The AI agent market is projected to reach **$7.38 billion by end of 2025**, nearly doubling from 2023
- **85% of organizations** have integrated AI agents in at least one workflow
- **82% plan full integration by 2026**

**🔑 Key Technology Trends**

1. **Autonomous Task Execution** - AI agents are becoming the "apps of the AI era," handling complex workflows independently

2. **Multi-Agent Systems** - Multiple specialized agents working together in "orchestrated" teams, each handling specific tasks

3. **Agentic RAG** - Combining retrieval-augmented generation with goal-driven autonomy, memory, and planning capabilities

4. **Advanced Reasoning** - Agents can now solve complex problems with logical steps similar to human thinking

5. **Voice-Controlled Agents** - Seamless integration of speech technologies with


## What's happening here:

- `query()` creates a single-turn agent interaction (no conversation memory)
- `allowed_tools=["WebSearch"]` gives Claude permission to search the web without asking for approval
- The agent autonomously decides when to search, what queries to run, and how to synthesize results
- `print_activity()` and `print_final_result` are helper functions that show the agent's actions in real-time and print the agent's final response along with cost and duration information.

That's it! A functional research agent in 10 lines of code. The agent will search for relevant information, follow up on promising leads, and provide a synthesized summary.

The query() function creates a stateless agent interaction. Each call is independent—no conversation memory, no context from previous queries. This makes it perfect for one-off research tasks where you need a quick answer without maintaining state.

**How tool permissions work:**

The `allowed_tools=["WebSearch"]` parameter gives Claude permission to search without asking for approval. This is critical for autonomous operation:

- `Allowed tools` - Claude can use these freely (in this case, WebSearch)
- `Other tools` - Available but require approval before use
- `Read-only tools` - Tools like Read are always allowed by default
- `Disallowed tools` - Add tools to disallowed_tools to remove them entirely from Claude's context

**When to use stateless queries:**

- One-off research questions where context doesn't matter
- Parallel processing of independent research tasks
- Scenarios where you want fresh context for each query

**When not to use stateless queries:**

- Multi-turn investigations that build on previous findings
- Iterative refinement of research based on initial results
- Complex analysis requiring sustained context

Let's inspect what the agent actually did using the visualize_conversation helper:

In [5]:
from utils.agent_visualizer import visualize_conversation

visualize_conversation(messages)


🤖 AGENT CONVERSATION TIMELINE

⚙️  System Initialized
   Session: 2b50cac5...

🤖 Assistant:
   💬 I'll research the latest trends in AI agents for you.

🤖 Assistant:
   🔧 Using tool: WebSearch
      Query: "latest trends AI agents 2025"

🤖 Assistant:
   🔧 Using tool: WebSearch
      Query: "AI agent developments recent advances"



🤖 Assistant:
   💬 ## Brief Summary: Latest Trends in AI Agents

Based on the latest research, here are the key trends shaping AI agents in 2025:

**🚀 Market Growth & Adoption**
- The AI agent market is projected to reach **$7.38 billion by end of 2025**, nearly doubling from 2023
- **85% of organizations** have integrated AI agents in at least one workflow
- **82% plan full integration by 2026**

**🔑 Key Technology Trends**

1. **Autonomous Task Execution** - AI agents are becoming the "apps of the AI era," handl...

✅ Conversation Complete
   Turns: 3
   Cost: $0.18
   Duration: 37.56s
   Tokens: 1,672




## From Prototype to Production: Three Key Improvements

Our one-line research agent works, but it's limited. Single queries without memory can't handle iterative research ("find X, then analyze Y based on what you found"). Let's explore three ways we can further improve our implementation.

**1. Conversation Memory with ClaudeSDKClient**: Stateless queries can't build on previous findings. If you ask "What are the top AI startups?" then "How are they funded?", the second query has no context about which startups you mean. We can use `ClaudeSDKClient` to maintain conversation history across multiple queries.


**2. System Prompts for Specialized Behavior**: Research domains often have specific requirements. Financial analysis needs different rigor than tech news summaries. Use the system prompt to encode your research standards, preferred sources, or output format. See our [agent prompting guide](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents/prompts) for research-specific examples.

**3. Multimodal Research with the Read Tool**: Real research isn't just text. Market reports have charts, technical docs have diagrams, competitive analysis requires screenshot comparison. Enable the `Read` tool so Claude can analyze images, PDFs, and other visual content.

Let's implement these three changes for our research agent.

In [6]:
from claude_agent_sdk import ClaudeSDKClient

messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        cwd="research_agent",
        system_prompt="You are a research agent specialized in AI",
        allowed_tools=["WebSearch", "Read"],
    )
) as research_agent:
    await research_agent.query("Analyze the chart in research_agent/projects_claude.png")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

    await research_agent.query("Use a single websearch to investigate the insights from the chart.")
    async for msg in research_agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Using: Glob()
✓ Tool completed
🤖 Thinking...
🤖 Using: Read()
✓ Tool completed
🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...


This example combines all three improvements: conversation memory via ClaudeSDKClient, a system prompt for AI research specialization, and the Read tool for analyzing visual content.

In the first query call, the agent reads and analyzes a chart image using the Read tool. Next, the Agent searches the web for context about the chart's findings—and critically, it remembers what it saw in the chart from the first query

The system prompt instruction helps the agent focus on relevant industry context.

One key difference from the first example: The `async with ClaudeSDKClient()` context manager maintains conversation state. The second query inherits context from the first—the agent knows which chart and which insights to investigate.

In [7]:
visualize_conversation(messages)


🤖 AGENT CONVERSATION TIMELINE

⚙️  System Initialized
   Session: d2214a68...

🤖 Assistant:
   💬 I'll read and analyze the chart image for you.

🤖 Assistant:
   🔧 Using tool: Read


🤖 Assistant:
   💬 Let me search for this file to find its correct location.

🤖 Assistant:
   🔧 Using tool: Glob


🤖 Assistant:
   💬 Now let me read the chart image:

🤖 Assistant:
   🔧 Using tool: Read


🤖 Assistant:
   💬 ## Analysis of the Chart

This chart compares the **"Types of Projects in Claude.ai and Claude Code"** showing how users utilize each platform differently.

### Key Findings:

**Personal Projects** (Most popular overall)
- Claude Code: **36.0%** (highest single category)
- Claude.ai: **30.2%**
- Gap: 5.8 percentage points in favor of Claude Code

**Startup Work** (Second highest)
- Claude Code: **32.9%** 
- Claude.ai: **13.1%**
- Gap: 19.8 percentage points - the **largest difference** between p...

✅ Conversation Complete
   Turns: 4
   Cost: $0.04
   Duration: 24.88s
   Tokens: 721

⚙️  

## Building for Production

Jupyter notebooks are great for learning, but production systems need reusable modules. We've packaged the research agent into research_agent/agent.py with a clean interface:

### Core functions:

- `print_activity()` - Shows what the agent is doing in real-time
- `get_activity_text()` - Extract activity text for custom handlers, such as logging or monitoring
- `send_query()` - Main entry point for research queries with built-in activity display

This agent can now be used in any Python script!

For independent questions where conversation context doesn't matter:


In [8]:
from research_agent.agent import send_query

result = await send_query("What is the Claude Code SDK? Only do one websearch and be concise")
print(f"\nResult: {result}\n")

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...

Result: The **Claude Code SDK** (now renamed **Claude Agent SDK**) is a framework built on top of the agent harness that powers Claude Code. It provides production-ready building blocks for creating AI agents that go beyond coding tasks.

**Key capabilities:**
- Context management with automatic compaction
- Rich tool ecosystem (file operations, code execution, web search, MCP extensibility)
- Error handling, session management, and monitoring
- Optimized for Claude with automatic prompt caching

**Installation:**
- Python: `pip install claude-agent-sdk`
- TypeScript/JS: `npm install @anthropic-ai/claude-agent-sdk`

Originally designed for coding agents, it now supports diverse use cases including research, legal review, finance analysis, customer support, and more.



Now we test out a multi-turn conversation that reuses the same conversation:

In [9]:
result1 = await send_query("What is Anthropic? Only do one websearch and be concise")
print(f"\n-----\n\nInitial research: {result1}\n")

🤖 Thinking...
🤖 Using: WebSearch()
✓ Tool completed
🤖 Thinking...

-----

Initial research: Anthropic is an AI safety and research company founded in 2021 by former OpenAI members, including CEO Dario Amodei. It's best known for creating **Claude**, a family of large language models (like the one you're talking to now). The company focuses on building safe, reliable, and interpretable AI systems. As of September 2025, Anthropic is valued at over $183 billion and has received major investments from Amazon ($8 billion total) and Google ($2 billion). Their mission emphasizes responsible AI development with systems that are helpful, honest, and harmless.



In [10]:
# Continue the conversation to dig deeper by setting continue_conversation=True
result2 = await send_query(
    "What are some of their products?",
    continue_conversation=True,
)
print(f"\n-----\n\nFollow-up: {result2}\n")

🤖 Thinking...

-----

Follow-up: Based on the search I just performed, Anthropic's main product is **Claude** - a family of large language models that function as AI assistants. 

The Claude family includes different versions:
- **Claude Opus** - their most capable model
- **Claude Sonnet** - a balanced model for performance and speed
- **Claude Haiku** - their fastest model

In May 2025, they announced **Claude 4**, introducing Claude Opus 4 and Claude Sonnet 4 with improved coding capabilities and other enhancements.

Claude can perform various tasks including:
- Analysis and research
- Coding and software development
- Creative writing
- Answering questions and having conversations

The models are accessible through their website (claude.ai) and via API for developers to integrate into their own applications.



## Conclusion

### What You Built

In this cookbook, you built three progressively sophisticated research agents:

- Stateless research agent - One-line queries for independent research tasks
- Stateful agent with memory - Multi-turn investigations that build on previous findings
- Production module - Reusable research functions for integration into applications

### Key Takeaways

**When to use stateless queries (query()):**

- Independent research questions
- Parallel processing of unrelated tasks
- Scenarios requiring fresh context each time

**When to use stateful agents (ClaudeSDKClient):**

- Multi-turn investigations building on previous findings
- Iterative refinement of research
- Complex analysis requiring sustained context

Research agents excel when information isn't self-contained and the optimal workflow emerges during exploration—competitive analysis, technical troubleshooting, literature reviews, and investigative journalism all fit this pattern.

### Next Steps

This foundation in autonomous research prepares you for enterprise-grade multi-agent systems. In the next notebook, you'll learn to:

Orchestrate specialized subagents under a coordinating agent
Implement governance through hooks and custom commands
Adapt output styles for different stakeholders (executives vs. technical teams)

Next: [01_The_chief_of_staff_agent.ipynb](01_The_chief_of_staff_agent.ipynb) - From single agents to multi-agent orchestration.